In [1]:
import pandas as pd

import geopandas as gpd
from glob import glob

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
kdprov = '21'

In [4]:
# open data sp
data_peta_w_ruta = gpd.read_file("data/output/0_raster_data_desa/desa_pend_rasterize_"+kdprov+".gpkg").to_crs('ESRI:54009')#.area

#open built-up
data_builtup=gpd.read_file("data/input/GHSL Data/2020/gpkg/ghs_built_"+kdprov+"_nonzero.shp")#.to_crs(('ESRI:54009'))
data_builtup['DN']=data_builtup.value
#open grid
#grid=gpd.read_file('data/input/grid_'+kdprov+'.gpkg').reset_index()#.to_crs(('ESRI:54009'))

In [5]:
data_peta_w_ruta.pop2020.sum()

2059387

In [6]:
data_builtup['centroid'] = data_builtup.centroid
data_builtup['polygon'] = data_builtup['geometry']
data_builtup.crs='ESRI:54009'
data_peta_w_ruta.crs='ESRI:54009'
data_builtup = data_builtup.loc[:,['DN','centroid','polygon']].set_geometry("centroid").sjoin(data_peta_w_ruta)
data_builtup['TOTAL_PEND']=data_builtup['pop2020']
# data_builtup = data_builtup.loc[:,['DN','geometry']].sjoin(data_peta_w_ruta)
data_builtup.shape

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


(833755, 9)

In [7]:
data_builtup['p_area'] = data_builtup['DN_left'] / data_builtup.groupby('iddesa').DN_left.transform('sum')
data_builtup['j_pend'] = list(data_builtup['p_area']*data_builtup['TOTAL_PEND'])

# data_builtup[['DN_left', 'geometry', 'iddesa','TOTAL_PEND', 'p_area','j_pend']].to_file("data/input/grid_pop_"+kdprov+"_100m.gpkg", driver="GPKG")
data_builtup[['DN_left', 'centroid', 'iddesa','TOTAL_PEND', 'p_area','j_pend']].to_file("data/temp/grid_pop_"+kdprov+"_100m.gpkg", driver="GPKG")

In [8]:
data_=data_builtup[['DN_left', 'polygon', 'iddesa','TOTAL_PEND', 'p_area','j_pend']].set_geometry("polygon")
data_.crs='ESRI:54009'
data_.to_file("data/output/1_population_density/grid_pop_"+kdprov+"_100m.gpkg", driver="GPKG")

In [9]:
#data_builtup.index_right.nunique()
data_builtup.j_pend.sum()

2058442.9999999998

In [10]:
data_builtup

,DN_left,centroid,polygon,index_right,iddesa,DN_right,id,pop2020,TOTAL_PEND,p_area,j_pend
0,0,POINT (10350350.000 134750.000),"POLYGON ((10350400.000 134700.000, 10350300.00...",67,2101034004,67.0,67.0,3134,3134,0.0,0.0
1,0,POINT (10350350.000 134650.000),"POLYGON ((10350400.000 134600.000, 10350300.00...",67,2101034004,67.0,67.0,3134,3134,0.0,0.0
2,0,POINT (10350350.000 134550.000),"POLYGON ((10350400.000 134500.000, 10350300.00...",67,2101034004,67.0,67.0,3134,3134,0.0,0.0
3,0,POINT (10350450.000 135250.000),"POLYGON ((10350500.000 135200.000, 10350400.00...",67,2101034004,67.0,67.0,3134,3134,0.0,0.0
4,0,POINT (10350450.000 135150.000),"POLYGON ((10350500.000 135100.000, 10350400.00...",67,2101034004,67.0,67.0,3134,3134,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
839586,0,POINT (10933750.000 320150.000),"POLYGON ((10933800.000 320100.000, 10933700.00...",195,2103062002,196.0,196.0,696,696,0.0,0.0
839587,0,POINT (10933750.000 320050.000),"POLYGON ((10933800.000 320000.000, 10933700.00...",195,2103062002,196.0,196.0,696,696,0.0,0.0
839588,0,POINT (10933750.000 319950.000),"POLYGON ((10933800.000 319900.000, 10933700.00...",195,2103062002,196.0,196.0,696,696,0.0,0.0
839589,0,POINT (10933750.000 319850.000),"POLYGON ((10933800.000 319800.000, 10933700.00...",195,2103062002,196.0,196.0,696,696,0.0,0.0
